# Puc Minas

## Pós Graduação em Ciência de Dados e Big Data (2020-2022)

## Trabalho de Conclusão de Curso 

### Aluna: Verônica Sá Vieira da Silva



O presente Trabalho de Conclusão de Curso em Ciências de dados e Big data estuda as Rodovias do Brasil 
utilizando o dataset do site <a href="https://dados.antt.gov.br/dataset?res_format=CSV&license_id=cc-by&res_format=JSON&groups=rodovias&page=2">ANTT</a>.


### Importação das bibliotecas utilizadas:


In [3]:
# -*- coding: utf-8 -*-
### Bibliotecas
import os 
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt


### Coleta de Dados:

Dando início à coleta de dados, as informações foram extraídas dos arquivos disponibilizados pela ANTT referente à acidentes, rodovias, postos das rodovias federais, informações dos terrenos e dos radares. 
Cada dataset foi separado por entidades para melhor manuseio das informações das rodovias.

In [4]:
DIRETORIO = os.path.join(os.getcwd(),'..','files')


### Acidentes
# Foi necessário agrupar 24 arquivos CSV pois a ANTT dividiu todos os acidentes por concessionária.
all_files = glob.glob(os.path.join(DIRETORIO,'acidentes_antt') + "/*.csv")

li = []
#columns_types = {'data': str, 'n_da_ocorrencia': str, 'tipo_de_ocorrencia': str, 'trecho': str, 'tipo_de_acidente': str}

for filename in all_files:
    
    df = pd.read_csv(filename, index_col=None, header=0, delimiter = ";", encoding="iso8859-1", low_memory = False)
    li.append(df)

df_acidentes = pd.concat(li, axis=0, ignore_index=True)

### Radar
df_radar = pd.read_csv(os.path.join(DIRETORIO,'radar','dados_dos_radares.csv'), encoding="iso8859-1",delimiter = ";", low_memory = False)

### Terreno
df_terreno = pd.read_csv(os.path.join(DIRETORIO,'terreno','dados_perfil_do_terreno.csv'), encoding="iso8859-1",delimiter = ";", low_memory = False)

### Postos PRF
df_postos_prf = pd.read_csv(os.path.join(DIRETORIO,'posto_prf','dados_dos_postos_prfs.csv'), encoding="iso8859-1",delimiter = ";", low_memory = False)



### Tratamento de Dados:

1. Tratando Inconsistências:

Pela natureza do dataset, muitos dados são manuais e precisam se tratados e agrupados. Foram criados novas colunas no dataset de Acidentes e no data set de Terrenos (UF e Rodovia) e foram feitos replaces na coluna de TIPO_DE_OCORRENCIA dos acidentes que descreve o tipo de acidente ocorrido. Muitas descrições não estão padronizadas, essa sessão tem o intuito de remover as inconsistências causadas por dados manuais.

In [5]:
## Criar as colunas Rodovia e UF no dataset de Acidentes.
df_acidentes['rodovia'] = df_acidentes.trecho
df_acidentes[['rodovia','uf']] = df_acidentes['rodovia'].str.split('/',expand=True)

## Criar as colunas Rodovia e UF no dataset de Terrenos.
df_terreno['rodovia'] = df_terreno.rodovia_uf
df_terreno[['rodovia','uf']] = df_terreno['rodovia'].str.split('/',expand=True)


### Replace:
# Rodovias:
df_acidentes['rodovia'].replace('Retão', 'BR-116',inplace=True)
df_acidentes['rodovia'].replace('Contorno', 'BR-101',inplace=True)
df_acidentes['rodovia'].replace('CONTORNO', 'BR-101',inplace=True)
df_terreno['rodovia'].replace('Contorno','BR-101',inplace=True)
df_terreno['rodovia'].replace('Contorno de Iconha','BR-101',inplace=True)
df_terreno['rodovia'].replace('Contorno de Uberlândia','BR-365',inplace=True)
df_terreno['rodovia'].replace('CONTORNO NORTE  - FS - 324','BR-324',inplace=True)


# Tipo de Ocorrencia:
df_acidentes['tipo_de_ocorrencia'].replace('sem vítima', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente sem vítima', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ASV - Acidente SEM Vítima', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac03 - Acidente sem Vitimas', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Sem vítima', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente sem Vítimas', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente sem Vítima', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente sem VITIMA', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC3 - Acidente sem VITIMA', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente sem vítima - TRECHO CONTORNO', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente sem Vítima - TRECHO CONTORNO', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente sem Vítima', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC03 - ACIDENTE SEM VÍTIMA', 'Acidente sem vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC03 - Acidente sem VITIMA', 'Acidente sem vitima',inplace=True)


df_acidentes['tipo_de_ocorrencia'].replace('com vítima', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com vítima', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('2 - Acidente com VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AV - Acidente com VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac02 - Acidente com VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Com vítima', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente COM VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com Vítimas', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('2- Acidente com VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac02 - Acidente com Vítima', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace(' 2- Acidente com VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac03 - Acidente com vítima ilesa', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('2 - Acidente com Vitima', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com vítima - TRECHO CONTORNO', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com Vítima - TRECHO CONTORNO', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC02 - ACIDENTE COM VÍTIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('3 - Acidente com Vitima Sem Lesão', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC02 - Acidente com VITIMA', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac02 - Acidente com vítima', 'Acidente com vitima',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC2 - Acidente com VITIMA', 'Acidente com vitima',inplace=True)



df_acidentes['tipo_de_ocorrencia'].replace('1 - Acidente com Vitima Fatal', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AF - Acidente com Vitima Fatal', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com morte', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac01 - Acidente com Vitima Fatal', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente Fatal', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente fatal', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com morte - TRECHO CONTORNO', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac01 - Acidente com vitima fatal', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC1 - Acidente com VITIMA FATAL', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com VITIMA FATAL', 'Acidente com Vitima Fatal',inplace=True)

df_acidentes['tipo_de_ocorrencia'].replace('Acidente com Morte - TRECHO CONTORNO', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Acidente com Morte', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC01 - ACIDENTE COM VÍTIMA FATAL', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC01 - Acidente com VITIMA FATAL', 'Acidente com Vitima Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('ac01 - Acidente com Vítima Fatal', 'Acidente com Vitima Fatal',inplace=True)


df_acidentes['tipo_de_ocorrencia'].replace('3 - Acidente com Danos Materiais', 'Acidente com Danos Materiais',inplace=True)

df_acidentes['tipo_de_ocorrencia'].replace('Atropelamento com morte', 'Atropelamento Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Atropelamento com Morte', 'Atropelamento Fatal',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC05 - Atropelamento Fatal', 'Atropelamento Fatal',inplace=True)

df_acidentes['tipo_de_ocorrencia'].replace('Atropelamento sem Morte - TRECHO CONTORNO', 'Atropelamento',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('AC04 - Atropelamento', 'Atropelamento',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Atropelamento sem morte', 'Atropelamento',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Atropelamento sem Morte', 'Atropelamento',inplace=True)
df_acidentes['tipo_de_ocorrencia'].replace('Atropelamento sem morte - TRECHO CONTORNO', 'Atropelamento',inplace=True)


df_acidentes['tipo_de_ocorrencia'].replace('in21 - Suícidio', 'Suícidio',inplace=True)

df_acidentes['tipo_de_ocorrencia'].replace('A PP - Acidente envolvendo Produto Perigoso', 'Acidente envolvendo Produto Perigoso',inplace=True)


df_acidentes['tipo_de_ocorrencia'].replace('A PPV - Acidente envolvendo Produto Perigoso com vitima', 'Acidente envolvendo Produto Perigoso com vitima',inplace=True)

df_acidentes['tipo_de_ocorrencia'].replace('A PPF - Acidente envolvendo Produto Perigoso vitima fatal', 'Acidente envolvendo Produto Perigoso vitima fatal',inplace=True)

df_acidentes['tipo_de_ocorrencia'].replace('ac03 - Acidente com Danos Materiais', 'Acidente com Danos Materiais',inplace=True)
 
## Verificando os valores únicos para TIPO_DE_OCORRENCIA
# print(df_acidentes['tipo_de_ocorrencia'].unique())

2. Removendo valores nulos

Removendo valores nulos e preenchendo com valores relevantes para o projeto.

In [6]:
# Identificando valores Nulos em colunas - Acidentes
df_acidentes.isna().any()

data                              False
horario                           False
n_da_ocorrencia                   False
tipo_de_ocorrencia                 True
km                                 True
trecho                            False
sentido                            True
tipo_de_acidente                   True
automovel                          True
bicicleta                          True
caminhao                           True
moto                               True
onibus                             True
outros                             True
tracao_animal                      True
transporte_de_cargas_especiais     True
trator_maquinas                    True
utilitarios                        True
ilesos                             True
levemente_feridos                  True
moderadamente_feridos              True
gravemente_feridos                 True
mortos                             True
rodovia                           False
uf                                 True


In [7]:
# Preenchendo os valores nulos
df_acidentes.data.fillna('00-00-000', inplace=True)
df_acidentes.horario.fillna('N/A', inplace=True)
df_acidentes.n_da_ocorrencia.fillna(0, inplace=True)
df_acidentes.tipo_de_ocorrencia.fillna('Não Informado', inplace=True)
df_acidentes.km.fillna(0, inplace=True)
df_acidentes.trecho.fillna('Não Informado', inplace=True)
df_acidentes.sentido.fillna('N/A', inplace=True)
df_acidentes.tipo_de_acidente.fillna('Não Informado', inplace=True)
df_acidentes.automovel.fillna(0, inplace=True)
df_acidentes.bicicleta.fillna(0, inplace=True)
df_acidentes.caminhao.fillna(0, inplace=True)
df_acidentes.moto.fillna(0, inplace=True)
df_acidentes.onibus.fillna(0, inplace=True)
df_acidentes.outros.fillna(0, inplace=True)
df_acidentes.tracao_animal.fillna(0, inplace=True)
df_acidentes.automovel.fillna(0, inplace=True)
df_acidentes.transporte_de_cargas_especiais.fillna(0, inplace=True)
df_acidentes.trator_maquinas.fillna(0, inplace=True)
df_acidentes.utilitarios.fillna(0, inplace=True)
df_acidentes.ilesos.fillna(0, inplace=True)
df_acidentes.levemente_feridos.fillna(0, inplace=True)
df_acidentes.moderadamente_feridos.fillna(0, inplace=True)
df_acidentes.gravemente_feridos.fillna(0, inplace=True)
df_acidentes.mortos.fillna(0, inplace=True)
df_acidentes.uf.fillna('N/A',inplace=True)

In [111]:
# Identificando valores Nulos em colunas - Radar
df_radar.isna().any()

concessionaria        False
ano_do_pnv_snv        False
tipo_de_radar         False
rodovia               False
uf                    False
km_m                  False
municipio             False
tipo_pista            False
sentido               False
situacao              False
data_da_inativacao    False
latitude              False
longitude             False
velocidade_leve       False
velocidade_pesado     False
dtype: bool

In [8]:
# Preenchendo os valores nulos
df_radar.data_da_inativacao.fillna('00-00-000', inplace=True)

In [113]:
# Identificando valores Nulos em colunas - Terreno
df_terreno.isna().any()

concessionaria            False
ano_do_pnv_snv            False
rodovia_uf                False
tipo_pista                False
sentido                   False
tipo_perfil_de_terreno    False
km_m_inicial              False
latitude_inicial          False
longitude_inicial         False
latitude_final            False
km_m_final                False
longitude_final           False
rodovia                   False
uf                        False
dtype: bool

In [114]:
# Identificando valores Nulos em colunas - Postos PRF
df_postos_prf.isna().any()

concessionaria        False
nome_posto_prf        False
ano_do_pnv_snv        False
rodovia               False
uf                    False
km_m                  False
municipio             False
tipo_pista            False
sentido               False
situacao              False
data_da_inativacao    False
latitude              False
longitude             False
dtype: bool

In [9]:
# Preenchendo os valores nulos
df_postos_prf.data_da_inativacao.fillna('00-00-000', inplace=True)

In [116]:
# Removendo duplicadas considerando todas as colunas

df_acidentes.drop_duplicates()
df_radar.drop_duplicates()
df_terreno.drop_duplicates()
df_postos_prf.drop_duplicates()


,concessionaria,nome_posto_prf,ano_do_pnv_snv,rodovia,uf,km_m,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude
0,AUTOPISTA FLUMINENSE,UOP 01 da 5ª DEL,2019,BR-101,RJ,"27,800",Campos dos Goytacazes,Principal,Decrescente,Ativo,00-00-000,"-21,447613","-41,343120"
1,AUTOPISTA FLUMINENSE,UOP 02 da 5ª DEL,2019,BR-101,RJ,"78,150",Campos dos Goytacazes,Principal,Decrescente,Ativo,00-00-000,"-21,835839","-41,420620"
2,AUTOPISTA FLUMINENSE,UOP 04 da 2ª DEL,2019,BR-101,RJ,"203,370",Casimiro de Abreu,Principal,Crescente,Ativo,00-00-000,"-22,475444","-42,172346"
3,AUTOPISTA FLUMINENSE,UOP 03 da 2ª DEL,2019,BR-101,RJ,"269,050",Rio Bonito,Principal,Crescente,Ativo,00-00-000,"-22,715042","-42,679022"
4,AUTOPISTA FLUMINENSE,UOP 02 da 2ª DEL,2019,BR-101,RJ,"308,070",São Gonçalo,Principal,Crescente,Ativo,00-00-000,"-22,792949","-43,024864"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,VIA BAHIA,Poções,2003,BR-116,BA,"759,000",Poções,Principal,Decrescente,Ativo,00-00-000,"-14,543002","-40,395276"
74,VIA BAHIA,Vitória da Conquista,2003,BR-116,BA,"830,000",Vitória da Conquista,Principal,Decrescente,Ativo,00-00-000,"-14,912414","-40,875554"
75,VIA BAHIA,Divisa BA/MG,2003,BR-116,BA,"931,700",Encruzilhada,Principal,Crescente,Ativo,00-00-000,"-15,685175","-41,325423"
76,VIA SUL,Polícia Rodoviária Federal de Torres,2019,BR-101,RS,"0,200",Torres,Principal,Crescente/Decrescente,Ativo,00-00-000,"-29,300935","-49,771369"


3. Exclusão de registros

Neste trabalho iremos estudar apenas as rodovias federais do Brasil. Foram encontrados nos dados rodovias fora do escopo.
Por esse motivo, o código abaixo filtra  todos os datasets apenas pelas rodovias federais do Brasil.

In [10]:
df_acidentes = df_acidentes[df_acidentes['rodovia'].str.contains("BR")]
df_terreno = df_terreno[df_terreno['rodovia'].str.contains("BR")]
df_postos_prf = df_postos_prf[df_postos_prf['rodovia'].str.contains("BR")]

Outro filtro necessário são os radares e postos da Polícia Federal ativos:

In [11]:
df_radar = df_radar[df_radar['situacao'].str.contains("Ativo")]
df_postos_prf = df_postos_prf[df_postos_prf['situacao'].str.contains("Ativo")]

### Análise e Exploração dos Dados

Quantidade de Acidentes por cada Rodovia

In [134]:
df_acidentes.groupby(['rodovia']).size().reset_index(name='acidentes').sort_values(by=['acidentes'], ascending=False)


,rodovia,acidentes
5,BR-116,250133
4,BR-101,163080
15,BR-381,100819
0,BR-040,80583
6,BR-153,25867
7,BR-163,23721
14,BR-376,22556
11,BR-324,21257
8,BR-262,13698
12,BR-364,10651



Quantidade de acidentes com vítima fatal

In [135]:
df_acidentes[df_acidentes['mortos'] != 0].groupby(['rodovia']).size().reset_index(name='mortes').sort_values(by=['mortes'], ascending=False)

,rodovia,mortes
5,BR-116,5460
4,BR-101,3071
15,BR-381,1657
0,BR-040,1490
6,BR-153,650
7,BR-163,607
11,BR-324,434
8,BR-262,420
12,BR-364,325
14,BR-376,269


Radar por Rodovia.

In [137]:
df_radar[df_radar['situacao'] == 'Ativo'].groupby(['rodovia']).size().reset_index(name='radar').sort_values(by=['radar'], ascending=False)

,rodovia,radar
0,BR-101,118
1,BR-116,101
2,BR-40,36
4,BR-60,36
3,BR-50,16
5,BR-70,7


Postos da Policia federal por Rodovia.

In [139]:
df_postos_prf[df_postos_prf['situacao'] == 'Ativo'].groupby(['rodovia']).size().reset_index(name='postos_prf').sort_values(by=['postos_prf'], ascending=False)

,rodovia,postos_prf
1,BR-116,37
0,BR-101,24
2,BR-40,8
3,BR-50,4
4,BR-60,3


Tipo de veículo envolvido nos acidentes

Tipo de acidentes por rodovia

In [149]:
df_acidentes.groupby(['rodovia','tipo_de_ocorrencia']).size().reset_index(name='qtd_ocorrencias').sort_values(by=['qtd_ocorrencias'], ascending=False)

,rodovia,tipo_de_ocorrencia,qtd_ocorrencias
29,BR-116,Acidente sem vitima,156810
20,BR-101,Acidente sem vitima,98875
25,BR-116,Acidente com vitima,85410
69,BR-381,Acidente sem vitima,63459
19,BR-101,Acidente com vitima,54644
...,...,...,...
55,BR-324,Acidente envolvendo Produto Perigoso com vitima,2
54,BR-324,Acidente envolvendo Produto Perigoso,2
28,BR-116,Acidente envolvendo Produto Perigoso vitima fatal,2
27,BR-116,Acidente envolvendo Produto Perigoso com vitima,1


Tipo de terreno por rodovia

In [152]:
df_terreno.groupby(['rodovia','tipo_perfil_de_terreno']).size().reset_index(name='perfil_terreno').sort_values(by=['perfil_terreno'], ascending=False)

,rodovia,tipo_perfil_de_terreno,perfil_terreno
4,BR-116,Perfil Ondulado,3561
2,BR-101,Perfil Plano,1789
10,BR-50,Perfil Montanhoso,1450
5,BR-116,Perfil Plano,1120
11,BR-50,Perfil Ondulado,1071
12,BR-50,Perfil Plano,967
1,BR-101,Perfil Ondulado,922
3,BR-116,Perfil Montanhoso,860
0,BR-101,Perfil Montanhoso,181
7,BR-365,Perfil Montanhoso,110


Correlação dos Radares ativos nas rodovias com maior quantidade de acidente

In [14]:
df_terreno.head()

inner_merged_total = pd.merge(df_terreno, df_acidente, on=["rodovia", "uf"], how='inner')

,concessionaria,ano_do_pnv_snv,rodovia_uf,tipo_pista,sentido,tipo_perfil_de_terreno,km_m_inicial,latitude_inicial,longitude_inicial,latitude_final,km_m_final,longitude_final,rodovia,uf
0,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,112","-19,962523","-44,157044","-19,962813","0,161","-44,157272",BR-101,MG
1,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,162","-19,962813","-44,157272","-19,963102","0,191","-44,157501",BR-101,MG
2,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,192","-19,963102","-44,157501","-19,963682","0,281","-44,157957",BR-101,MG
3,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,282","-19,963682","-44,157957","-19,963828","0,291","-44,158070",BR-101,MG
4,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,292","-19,963828","-44,158070","-19,964427","0,384","-44,158498",BR-101,MG


Correlação dos Terrenos nas rodovias com maior quantidade de acidente

,concessionaria,ano_do_pnv_snv,rodovia_uf,tipo_pista,sentido,tipo_perfil_de_terreno,km_m_inicial,latitude_inicial,longitude_inicial,latitude_final,km_m_final,longitude_final,rodovia,uf
0,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,112","-19,962523","-44,157044","-19,962813","0,161","-44,157272",BR-101,MG
1,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,162","-19,962813","-44,157272","-19,963102","0,191","-44,157501",BR-101,MG
2,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,192","-19,963102","-44,157501","-19,963682","0,281","-44,157957",BR-101,MG
3,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,282","-19,963682","-44,157957","-19,963828","0,291","-44,158070",BR-101,MG
4,AUTOPISTA FERNÃO DIAS,2007,Contorno/MG,Principal,Crescente,Perfil Montanhoso,"0,292","-19,963828","-44,158070","-19,964427","0,384","-44,158498",BR-101,MG


Testes de hipotese e intervalos de confianca

### Criação, treinamento, aplicação e avaliação de modelos Machine Learning

### Interpretação dos Resultados

### Apresentacao dos Resultados